<a href="https://colab.research.google.com/github/HenryLimaa/Jobs/blob/main/DesafioPP_AB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio de Programação: Classificação de Pistas de Pouso em Imagens de Satélite**

## **Abordagem: Utilizando Google Earth Engine**

## I. Análise Simplificada

In [71]:
# @title INSTALAÇÃO DE BIBLIOTECAS

!pip install earthengine-api
!pip install geemap tensorflow rasterio matplotlib scikit-learn numpy ipywidgets
!pip install segmentation_models_pytorch > /dev/null
!pip install sentinelhub > /dev/null
!pip install geopandas > /dev/null
!pip install shapely > /dev/null
!pip install scikit-image > /dev/null
!pip install --upgrade ipywidgets > /dev/null
!pip install folium > /dev/null
!pip install --upgrade pip > /dev/null
!pip install ipywidgets[widgetsnbextension] > /dev/null
!pip install --upgrade ipywidgets > /dev/null



print("Bibliotecas instaladas!")

Bibliotecas instaladas!


In [72]:
# @title 1 - Carregar Bibliotecas

# EARTH ENGINE & GEOESPACIAL
import ee
import geemap

# DEEP LEARNING & IA
import tensorflow as tf
from tensorflow.keras import layers, models

# PROCESSAMENTO DE IMAGENS
import numpy as np
import cv2
import rasterio
from rasterio.transform import from_origin

# VISUALIZAÇÃO
import matplotlib.pyplot as plt

# PRÉ-PROCESSAMENTO & VALIDAÇÃO
from sklearn.model_selection import train_test_split

# MANIPULAÇÃO DE ARQUIVOS & DOWNLOAD
import requests
import zipfile
import os
from io import BytesIO

# UTILITÁRIOS
import datetime




import ipywidgets as widgets
from IPython.display import display
import folium

In [73]:
# @title AUTENTICAÇÃO GOOGLE EARTH ENGINE

try:
    ee.Initialize(project='your-earthengine-project-id')
    print("Google Earth Engine inicializado!")
except Exception as e:
    print("Autenticando no Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project='my-earth-engine-project-472319')
    print("Google Earth Engine inicializado após autenticação!")

Google Earth Engine inicializado!


In [74]:
# @title CONFIGURAÇÃO COM 150KM DE RAIO

# Coordenadas precisas do Aeroporto de Altamira
altamira_airport = [-52.2540, -3.2530]

# Área de 150km de raio (área muito grande - 70,650 km²)
roi_150km = ee.Geometry.Point(altamira_airport).buffer(150000)  # 150km
roi_50km = ee.Geometry.Point(altamira_airport).buffer(50000)    # 50km
roi_10km = ee.Geometry.Point(altamira_airport).buffer(10000)    # 10km
roi_5km = ee.Geometry.Point(altamira_airport).buffer(5000)      # 5km
roi_2km = ee.Geometry.Point(altamira_airport).buffer(2000)      # 2km para teste rápido

print("Aeroporto de Altamira:", altamira_airport)
print("Área de análise: 150km de raio")
print("Área total: {:,} km²".format(int(3.14 * 150 * 150)))
print("Abrange múltiplos municípios do Pará")

Aeroporto de Altamira: [-52.254, -3.253]
Área de análise: 150km de raio
Área total: 70,650 km²
Abrange múltiplos municípios do Pará


In [75]:
# @title FUNÇÃO OTIMIZADA PARA IMAGENS (150KM) COM NDVI

def get_sentinel2_with_ndvi(roi):
    """Obtém imagem Sentinel-2 com cálculo de NDVI"""
    print("Buscando imagem Sentinel-2 para área de 150km...")

    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterBounds(roi)
                 .filterDate('2024-06-01', '2024-08-31')  # Estação seca
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
                 .sort('CLOUDY_PIXEL_PERCENTAGE')
                 .first())

    image = collection

    # Calcular NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Adicionar NDVI à imagem
    image_with_ndvi = image.addBands(ndvi)

    print("Imagem e NDVI calculados!")
    return image_with_ndvi

In [76]:
# @title VISUALIZAÇÃO DA ÁREA DE 150KM

def quick_area_preview():
    """Visualização da área de estudo de 150km"""
    print("\n VISUALIZAÇÃO DA ÁREA DE ESTUDO (150KM RAIO)")

    Map = geemap.Map(center=[-3.2530, -52.2540], zoom=8)

    # Adicionar áreas concêntricas
    Map.addLayer(roi_150km, {'color': 'red', 'fillColor': 'FF000010'}, 'Área 150km')
    Map.addLayer(roi_50km, {'color': 'orange', 'fillColor': 'FF800020'}, 'Área 50km')
    Map.addLayer(roi_10km, {'color': 'yellow', 'fillColor': 'FFFF0030'}, 'Área 10km')
    Map.addLayer(roi_5km, {'color': 'green', 'fillColor': '00FF0020'}, 'Área 5km')

    # Ponto do aeroporto
    airport_point = ee.Geometry.Point(altamira_airport)
    Map.addLayer(airport_point, {'color': 'purple', 'pointSize': 20}, 'Aeroporto Altamira')

    return Map

In [77]:
# @title MAPA INTERATIVO COMPLETO COM SENTINEL-2 RGB, NDVI E GOOGLE SATELLITE

def create_interactive_map_with_layers(image, roi):
    """Cria mapa interativo com layers Sentinel-2 RGB, NDVI e Google Satellite"""
    print("\n CRIANDO MAPA INTERATIVO COM LAYERS")

    Map = geemap.Map(center=[-3.2530, -52.2540], zoom=8)

    # LAYER 1: Google Satellite (Mapa de satélite do Google)
    print(" Adicionando layer Google Satellite...")
    Map.add_tile_layer(
        url='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        name='Google Satellite',
        attribution='Google',
        shown=True  # Visível por padrão
    )

    # LAYER 2: Sentinel-2 RGB (Banda True Color)
    print(" Adicionando layer Sentinel-2 RGB...")
    rgb_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.3
    }
    Map.addLayer(image.clip(roi), rgb_params, 'Sentinel-2 RGB', False)  # False = oculto por padrão

    # LAYER 3: NDVI (Normalized Difference Vegetation Index)
    print(" Adicionando layer NDVI...")
    ndvi_params = {
        'min': -1,
        'max': 1,
        'palette': ['blue', 'white', 'green', 'darkgreen']
    }
    Map.addLayer(image.select('NDVI').clip(roi), ndvi_params, 'NDVI - Vegetação', False)

    # LAYER 4: Áreas de estudo
    Map.addLayer(roi_150km, {'color': 'red', 'fillColor': '00000000'}, 'Área 150km', True)
    Map.addLayer(roi_50km, {'color': 'orange', 'fillColor': '00000000'}, 'Área 50km', False)
    Map.addLayer(roi_10km, {'color': 'yellow', 'fillColor': '00000000'}, 'Área 10km', False)

    # LAYER 5: Aeroporto e cidades
    airport_point = ee.Geometry.Point(altamira_airport)
    Map.addLayer(airport_point, {'color': 'purple', 'pointSize': 15}, 'Aeroporto Altamira', True)

    # Adicionar pontos das cidades na região
    cities_150km = {
        'Altamira': [-52.2540, -3.2530],
        'Anapu': [-51.1833, -3.4667],
        'Vitória do Xingu': [-51.9833, -2.8833],
        'Brasil Novo': [-52.4833, -3.5333],
        'Uruará': [-53.7500, -3.7167]
    }

    for city, coords in cities_150km.items():
        point = ee.Geometry.Point(coords)
        Map.addLayer(point, {'color': 'blue', 'pointSize': 8}, f'{city}', False)

    # LAYER 6: Detecção de pistas
    try:
        rgb = image.select(['B4', 'B3', 'B2'])
        brightness = rgb.reduce(ee.Reducer.mean()).rename('brightness')
        runway_candidates = brightness.gt(2000)
        Map.addLayer(runway_candidates.clip(roi),
                    {'palette': ['000000', '00FF00'], 'opacity': 0.6},
                    'Áreas Claras (Pistas)', False)
    except:
        pass

    # LAYER 7: Detecção avançada (NDVI + Brilho)
    try:
        enhanced_mask = enhanced_runway_detection(image, roi)
        Map.addLayer(enhanced_mask.clip(roi),
                   {'palette': ['000000', 'FF0000'], 'opacity': 0.7},
                   'Detecção Avançada (NDVI + Brilho)', False)
    except:
        pass

    # Adicionar controle de layers
    Map.addLayerControl()

    print(" Mapa interativo criado com sucesso!")
    print(" Layers disponíveis:")
    print("   • Google Satellite (VISÍVEL)")
    print("   • Sentinel-2 RGB (OCULTO)")
    print("   • NDVI - Vegetação (OCULTO)")
    print("   • Áreas de estudo")
    print("   • Cidades da região")
    print("   • Detecção de áreas claras")
    print("   • Detecção avançada")

    return Map

In [78]:
# @title ATUALIZAR MENSAGEM DE INSTRUÇÕES

def update_instructions():
    """Atualiza as instruções para incluir o Google Satellite"""
    print("\n COMO USAR O MAPA:")
    print("   • Use o controle no canto superior direito para ligar/desligar layers")
    print("   • Google Satellite: Mapa de satélite de alta resolução")
    print("   • Sentinel-2 RGB: Imagem atual da região")
    print("   • NDVI: Mostra vegetação (verde = muita vegetação)")
    print("   • Detecção Avançada: Áreas vermelhas = possíveis pistas")
    print("   • Dica: Compare Google Satellite com Sentinel-2 para análise temporal")

In [79]:
# @title FUNÇÃO PARA ANÁLISE DE NDVI
def analyze_ndvi(image, roi):
    """Análise específica do NDVI na região"""
    print("\n ANÁLISE DO NDVI NA REGIÃO DE 150KM")

    try:
        # Estatísticas do NDVI
        ndvi_stats = image.select('NDVI').reduceRegion(
            reducer=ee.Reducer.mean().combine(
                reducer2=ee.Reducer.stdDev(),
                sharedInputs=True
            ),
            geometry=roi,
            scale=1000,  # Escala maior para performance
            maxPixels=1e8,
            bestEffort=True
        )

        ndvi_mean = ndvi_stats.get('NDVI_mean').getInfo()
        ndvi_std = ndvi_stats.get('NDVI_stdDev').getInfo()

        if ndvi_mean is not None:
            print(f" Estatísticas do NDVI:")
            print(f"   • Média: {ndvi_mean:.3f}")
            print(f"   • Desvio padrão: {ndvi_std:.3f}")

            # Interpretação do NDVI
            if ndvi_mean > 0.6:
                veg_status = "Muita vegetação (floresta densa)"
            elif ndvi_mean > 0.3:
                veg_status = "Vegetação moderada"
            elif ndvi_mean > 0.1:
                veg_status = "Pouca vegetação (áreas abertas)"
            else:
                veg_status = "Solo exposto/área urbana"

            print(f"   • Status: {veg_status}")

            return ndvi_mean, ndvi_std
        else:
            print(" Não foi possível calcular estatísticas do NDVI")
            return None, None

    except Exception as e:
        print(f" Erro na análise do NDVI: {e}")
        return None, None

In [80]:
# @title DETECÇÃO DE PISTAS COM SUPORTE A NDVI
def enhanced_runway_detection(image, roi):
    """Detecção melhorada usando NDVI para excluir vegetação"""
    print(" Executando detecção avançada...")

    image_roi = image.clip(roi)

    # 1. Usar NDVI para remover vegetação
    ndvi = image_roi.select('NDVI')
    non_vegetation = ndvi.lt(0.3)  # Áreas com pouca vegetação

    # 2. Calcular brilho das áreas não vegetadas
    rgb = image_roi.select(['B4', 'B3', 'B2'])
    brightness = rgb.reduce(ee.Reducer.mean()).rename('brightness')

    # 3. Combinar condições: pouca vegetação + alta reflectância
    bright_areas = brightness.gt(1800)
    runway_candidates = non_vegetation.And(bright_areas)

    # 4. Limpeza morfológica
    cleaned = runway_candidates.focal_max(20).focal_min(10)

    return cleaned.rename('enhanced_runways')

In [81]:
# @title EXECUÇÃO PRINCIPAL COM LAYERS COMPLETOS

def main_with_layers():
    """Execução principal com mapa interativo completo"""
    print("=== DETECÇÃO DE PISTAS - MAPA INTERATIVO COMPLETO ===")
    print(" Área de abrangência: 150km raio")
    print(" Layers: Sentinel-2 RGB + NDVI")
    print("="*60)

    try:
        # 1. Visualização inicial da área
        print("1.  Visualizando área de estudo...")
        area_map = quick_area_preview()
        display(area_map)

        # 2. Obter imagem com NDVI
        print("\n2.  Obtendo imagem Sentinel-2 com NDVI...")
        image_with_ndvi = get_sentinel2_with_ndvi(roi_150km)
        print(" Imagem processada com sucesso!")

        # 3. Análise do NDVI
        print("\n3.  Analisando vegetação na região...")
        ndvi_mean, ndvi_std = analyze_ndvi(image_with_ndvi, roi_150km)

        # 4. Criar mapa interativo completo
        print("\n4.  Criando mapa interativo com layers...")
        interactive_map = create_interactive_map_with_layers(image_with_ndvi, roi_150km)

        # 5. Detecção avançada
        print("\n5.  Executando detecção avançada...")
        enhanced_mask = enhanced_runway_detection(image_with_ndvi, roi_150km)

        # Adicionar layer de detecção avançada ao mapa
        interactive_map.addLayer(enhanced_mask.clip(roi_150km),
                               {'palette': ['000000', 'FF0000'], 'opacity': 0.7},
                               'Detecção Avançada (NDVI + Brilho)', False)

        print(" Camada de detecção avançada adicionada!")

        # 6. Exibir mapa final
        display(interactive_map)

        print("\n" + "="*60)
        print(" MAPA INTERATIVO COMPLETO CRIADO!")
        print("\n COMO USAR O MAPA:")
        print("   • Use o controle no canto superior direito para ligar/desligar layers")
        print("   • Sentinel-2 RGB: Visão natural da região")
        print("   • NDVI: Mostra vegetação (verde = muita vegetação)")
        print("   • Detecção Avançada: Áreas vermelhas = possíveis pistas")
        print("   • Zoom: Use scroll do mouse para aproximar")

        if ndvi_mean is not None:
            print(f"\n NDVI médio: {ndvi_mean:.3f} ({'alta' if ndvi_mean > 0.6 else 'média' if ndvi_mean > 0.3 else 'baixa'} vegetação)")

        return interactive_map, image_with_ndvi

    except Exception as e:
        print(f" Erro na execução: {e}")
        return None, None

In [82]:
# @title MAPA SIMPLES FOCADO NO AEROPORTO

def airport_focus_map():
    """Mapa focado no aeroporto com layers"""
    print("\n MAPA FOCADO NO AEROPORTO DE ALTAMIRA")

    try:
        # Imagem para área menor
        image = get_sentinel2_with_ndvi(roi_10km)

        # Mapa com zoom maior
        Map = geemap.Map(center=[-3.2530, -52.2540], zoom=12)

        # Layer 1: Sentinel-2 RGB
        Map.addLayer(image.clip(roi_10km),
                    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
                    'Sentinel-2 RGB (Zoom)', True)

        # Layer 2: NDVI
        Map.addLayer(image.select('NDVI').clip(roi_10km),
                    {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']},
                    'NDVI (Zoom)', False)

        # Layer 3: Aeroporto
        airport_point = ee.Geometry.Point(altamira_airport)
        Map.addLayer(airport_point, {'color': 'red', 'pointSize': 20}, 'Aeroporto', True)

        # Layer 4: Detecção
        detection = enhanced_runway_detection(image, roi_10km)
        Map.addLayer(detection,
                    {'palette': ['000000', 'FF0000'], 'opacity': 0.8},
                    'Detecção (Zoom)', False)

        Map.addLayerControl()

        print(" Mapa focado criado!")
        return Map

    except Exception as e:
        print(f" Erro no mapa focado: {e}")
        return None

# Criar mapa focado
print("\n Criando mapa focado no aeroporto...")
focus_map = airport_focus_map()
if focus_map:
    display(focus_map)
    print(" Análise detalhada da regiao do aeroporto")


 Criando mapa focado no aeroporto...

 MAPA FOCADO NO AEROPORTO DE ALTAMIRA
Buscando imagem Sentinel-2 para área de 150km...
Imagem e NDVI calculados!
 Erro no mapa focado: Project 'projects/your-earthengine-project-id' not found or deleted.


## II. Análise Detalhada

In [83]:
# Inicializar Earth Engine
try:
    # Replace 'your-earthengine-project-id' with your actual Earth Engine project ID
    ee.Initialize(project='my-earth-engine-project-472319')
except:
    ee.Authenticate()
    # Replace 'your-earthengine-project-id' with your actual Earth Engine project ID
    ee.Initialize(project='my-earth-engine-project-472319')

In [84]:
# @title # Filtragem de imagens Sentinel-2 para a região de interesse

def get_sentinel2_collection(roi, start_date, end_date, cloud_cover=20):
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(roi)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover))
                  .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']))  # RGB + NIR + SWIR
    return collection

In [85]:
# @title 2. Extração de Características para Detecção de Pistas

def extract_runway_features(image):
    # Índices espectrais relevantes para infraestrutura
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    # Textura para detectar padrões lineares (pistas)
    texture = image.select('B8').entropy(ee.Kernel.square(3)).rename('texture')

    return image.addBands([ndvi, ndbi, texture])

In [86]:
# @title 3. Algoritmo de Detecção de Pistas

def detect_runways(image):
    # Máscara para áreas urbanas/infraestrutura (baixa vegetação)
    urban_mask = image.select('NDVI').lt(0.2)

    # Detecção de features lineares (possibles pistas) using Sobel filter
    # Define Sobel kernels.
    sobel_h = ee.Kernel.fixed(3, 3,
                           [[1, 2, 1],
                            [0, 0, 0],
                            [-1, -2, -1]])

    sobel_v = ee.Kernel.fixed(3, 3,
                           [[1, 0, -1],
                            [2, 0, -2],
                            [1, 0, -1]])

    # Convolve the image with the Sobel kernels and compute the magnitude.
    edges_h = image.select('B8').convolve(sobel_h)
    edges_v = image.select('B8').convolve(sobel_v)
    edges = edges_h.pow(2).add(edges_v.pow(2)).sqrt().rename('edges')

    # Combinação de critérios
    runway_probability = (urban_mask
                         .multiply(edges.gt(100)) # Adjust threshold as needed
                         .multiply(image.select('NDBI').gt(0)))

    return runway_probability.rename('runway_prob')

In [87]:
# @title 4. Visualização dos Resultados no Mapa

def create_results_map(roi, start_date, end_date, cloud_cover=20):
    """
    Cria mapa interativo com os resultados da detecção de pistas
    """
    # Criar mapa centrado na ROI
    Map = geemap.Map(center=[roi.centroid().coordinates().get(1).getInfo(),
                            roi.centroid().coordinates().get(0).getInfo()],
                    zoom=12)

    # Obter coleção de imagens
    collection = get_sentinel2_collection(roi, start_date, end_date, cloud_cover)

    # Obter imagem mais recente com menos nuvens
    image = collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()

    # Extrair características
    image_with_features = extract_runway_features(image)

    # Detectar pistas
    runway_detection = detect_runways(image_with_features)

    # Adicionar capas ao mapa
    # 1. Imagem RGB natural
    visualization_rgb = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }
    Map.addLayer(image, visualization_rgb, 'Imagem Sentinel-2 RGB')

    # 2. Imagem Falsa Cor (Infravermelho)
    visualization_false_color = {
        'bands': ['B8', 'B4', 'B3'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }
    Map.addLayer(image, visualization_false_color, 'Falsa Cor (NIR-R-G)')

    # 3. Detecção de pistas (probabilidade)
    runway_vis_params = {
        'min': 0,
        'max': 1,
        'palette': ['black', 'blue', 'cyan', 'green', 'yellow', 'red']
    }
    Map.addLayer(runway_detection, runway_vis_params, 'Probabilidade de Pistas')

    # 4. Máscara de áreas de alta probabilidade
    high_prob_mask = runway_detection.gt(0.7)
    Map.addLayer(high_prob_mask, {'palette': ['black', 'red']}, 'Áreas de Alta Probabilidade (>70%)')

    # 5. Contornos das pistas detectadas
    contours = runway_detection.reduceToVectors(
        geometry=roi,
        scale=10,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='runway',
        maxPixels=1e10
    )
    Map.addLayer(contours.style(**{'color': 'yellow', 'fillColor': '00000000'}),
                {}, 'Contornos Detectados')

    # Converter ROI para FeatureCollection para estilizar e adicionar ao mapa
    roi_feature_collection = ee.FeatureCollection(roi)
    Map.addLayer(roi_feature_collection.style(**{'color': 'white', 'fillColor': '00000000'}), {}, 'Região de Interesse')


    return Map, image, runway_detection, contours

In [88]:
# @title 5. Painel de Controle Interativo

def create_analysis_panel():
    """
    Cria painel de controle para análise de detecção de pistas
    """
    import datetime

    # Widgets de entrada
    lat = widgets.FloatText(value=-23.5505, description='Latitude:')
    lon = widgets.FloatText(value=-46.6333, description='Longitude:')
    buffer = widgets.FloatSlider(value=5000, min=1000, max=10000, step=500,
                                description='Raio (m):')

    # Get Earth Engine dates as strings and convert to Python date objects
    start_date_str = ee.Date('2024-01-01').format().getInfo()
    end_date_str = ee.Date('2024-12-31').format().getInfo()

    # The format returned by ee.Date(...).format().getInfo() includes time, e.g., '2024-01-01T00:00:00'
    # We need to match this format or extract only the date part.
    # Let's adjust the format string to match the full string.
    start_date_obj = datetime.datetime.strptime(start_date_str, '%Y-%m-%dT%H:%M:%S').date()
    end_date_obj = datetime.datetime.strptime(end_date_str, '%Y-%m-%dT%H:%M:%S').date()


    start_date = widgets.DatePicker(description='Data Inicial:', value=start_date_obj)
    end_date = widgets.DatePicker(description='Data Final:', value=end_date_obj)

    cloud_cover = widgets.FloatSlider(value=20, min=0, max=100, step=5,
                                     description='Cobertura de Nuvens Máx (%):')

    threshold = widgets.FloatSlider(value=0.7, min=0, max=1, step=0.1,
                                   description='Limite de Probabilidade:')

    analyze_btn = widgets.Button(description='Executar Análise', button_style='success')
    output = widgets.Output()

    def on_analyze_click(b):
        with output:
            output.clear_output()
            print("🛰️ Iniciando análise de detecção de pistas...")

            # Criar região de interesse
            roi = ee.Geometry.Point([lon.value, lat.value]).buffer(buffer.value)

            # Executar análise
            Map, image, runway_detection, contours = create_results_map(
                roi,
                start_date.value.strftime('%Y-%m-%d'),
                end_date.value.strftime('%Y-%m-%d'),
                cloud_cover.value
            )

            # Exibir estatísticas
            stats = runway_detection.reduceRegion(
                reducer=ee.Reducer.mean().combine(
                    reducer2=ee.Reducer.max(),
                    sharedInputs=True
                ).combine(
                    reducer2=ee.Reducer.min(),
                    sharedInputs=True
                ),
                geometry=roi,
                scale=10,
                maxPixels=1e10
            )

            print("📊 Estatísticas da Detecção:")
            print(f"   • Probabilidade média: {stats.get('runway_prob_mean').getInfo():.3f}")
            print(f"   • Probabilidade máxima: {stats.get('runway_prob_max').getInfo():.3f}")
            print(f"   • Probabilidade mínima: {stats.get('runway_prob_min').getInfo():.3f}")

            # Contar áreas de alta probabilidade
            high_prob_mask = runway_detection.gt(threshold.value).multiply(ee.Image.pixelArea())
            total_high_prob = high_prob_mask.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=roi,
                scale=10,
                maxPixels=1e10
            )

            area_hectares = total_high_prob.get('runway_prob').getInfo() / 10000
            print(f"   • Área de alta probabilidade: {area_hectares:.2f} hectares")

            # Exibir mapa
            display(Map)

            print("✅ Análise concluída!")

    analyze_btn.on_click(on_analyze_click)

    # Layout do painel
    controls = widgets.VBox([
        widgets.HBox([lat, lon]),
        buffer,
        widgets.HBox([start_date, end_date]),
        cloud_cover,
        threshold,
        analyze_btn,
        output
    ])

    return controls

In [89]:
# @title 6. Função Principal da Aplicação

def create_runway_detection_app():
    """
    Cria aplicação completa de detecção de pistas de pouso
    """
    # Criar mapa inicial
    Map = geemap.Map(center=[-23.5505, -46.6333], zoom=10)

    # Adicionar camada base
    Map.add_basemap('SATELLITE')

    # Criar painel de controle
    control_panel = create_analysis_panel()

    # Layout da aplicação
    app_layout = widgets.AppLayout(
        center=Map,
        right_sidebar=control_panel,
        pane_widths=['70%', '0px', '30%'],
        height='800px'
    )

    return app_layout

In [90]:
# @title 7. Exemplo de Uso e Visualização

# Criar e exibir aplicação
print("🛫 Aplicação de Detecção de Pistas de Pouso")
print("=" * 50)

# Exemplo rápido para o Aeroporto de Altamira - Pará
print("\n📋 Exemplo Rápido: Aeroporto de Altamira (Pará)")

# Coordenadas do Aeroporto de Altamira: -3.2539, -52.2540
altamira_roi = ee.Geometry.Point([-52.2540, -3.2539]).buffer(3000)

# Executar análise rápida
quick_map, quick_image, quick_detection, quick_contours = create_results_map(
    altamira_roi,
    '2024-01-01',
    '2024-12-31',
    10
)

print("🗺️ Exibindo mapa do Aeroporto de Altamira...")
print("📍 Coordenadas: Lat -3.2539, Lon -52.2540")
display(quick_map)

# Criar aplicação interativa completa
#print("\n Iniciando aplicação interativa completa...")
#app = create_runway_detection_app()
#display(app)

🛫 Aplicação de Detecção de Pistas de Pouso

📋 Exemplo Rápido: Aeroporto de Altamira (Pará)
🗺️ Exibindo mapa do Aeroporto de Altamira...
📍 Coordenadas: Lat -3.2539, Lon -52.2540


Map(center=[-3.253896922334545, -52.253999914517934], controls=(WidgetControl(options=['position', 'transparen…

In [91]:
# @title 8. Análise de Resultados e Estatísticas

def generate_detection_report(roi, start_date, end_date):
    """
    Gera relatório detalhado da detecção
    """
    collection = get_sentinel2_collection(roi, start_date, end_date)
    image = collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()
    image_with_features = extract_runway_features(image)
    runway_detection = detect_runways(image_with_features)

    # Calcular estatísticas detalhadas
    stats = runway_detection.reduceRegion(
        reducer=ee.Reducer.histogram().combine(
            reducer2=ee.Reducer.mean(),
            sharedInputs=True
        ).combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ),
        geometry=roi,
        scale=10,
        maxPixels=1e10
    )

    print("📈 RELATÓRIO DE DETECÇÃO DE PISTAS")
    print("=" * 40)
    print(f"Período: {start_date} a {end_date}")
    print(f"Área analisada: {roi.area().getInfo() / 10000:.2f} hectares")
    print(f"\nEstatísticas de Probabilidade:")
    print(f"• Média: {stats.get('runway_prob_mean').getInfo():.3f}")
    print(f"• Desvio Padrão: {stats.get('runway_prob_stdDev').getInfo():.3f}")

    # Áreas por classe de probabilidade
    for threshold in [0.3, 0.5, 0.7, 0.9]:
        area = runway_detection.gt(threshold).multiply(ee.Image.pixelArea()) \
                              .reduceRegion(ee.Reducer.sum(), roi, 10) \
                              .get('runway_prob').getInfo() / 10000
        print(f"• Área > {threshold}: {area:.2f} ha")

# Exemplo de relatório para Altamira
print("\n📊 Gerando relatório do Aeroporto de Altamira...")
generate_detection_report(altamira_roi, '2024-01-01', '2024-12-31')


📊 Gerando relatório do Aeroporto de Altamira...
📈 RELATÓRIO DE DETECÇÃO DE PISTAS
Período: 2024-01-01 a 2024-12-31
Área analisada: 2793.65 hectares

Estatísticas de Probabilidade:
• Média: 0.023
• Desvio Padrão: 0.151
• Área > 0.3: 64.83 ha
• Área > 0.5: 64.83 ha
• Área > 0.7: 64.83 ha
• Área > 0.9: 64.83 ha


In [92]:
# @title 9. Análise Comparativa com Múltiplos Aeroportos

def compare_airports():
    """
    Compara a detecção em diferentes aeroportos brasileiros
    """
    airports = {
        'Altamira': {'coords': [-52.2540, -3.2539], 'buffer': 3000},
        'Belém': {'coords': [-48.4431, -1.3792], 'buffer': 3000},
        'Manaus': {'coords': [-60.0497, -3.0386], 'buffer': 4000},
        'Porto Velho': {'coords': [-63.9023, -8.7153], 'buffer': 3000}
    }

    print("🏢 ANÁLISE COMPARATIVA DE AEROPORTOS DA AMAZÔNIA")
    print("=" * 50)

    for name, info in airports.items():
        print(f"\n🔍 Analisando: {name}")
        roi = ee.Geometry.Point(info['coords']).buffer(info['buffer'])

        try:
            generate_detection_report(roi, '2024-01-01', '2024-12-31')
        except Exception as e:
            print(f"❌ Erro na análise de {name}: {e}")

# Executar análise comparativa (opcional)
print("\n🌎 Análise comparativa de aeroportos da região amazônica...")
compare_airports()


🌎 Análise comparativa de aeroportos da região amazônica...
🏢 ANÁLISE COMPARATIVA DE AEROPORTOS DA AMAZÔNIA

🔍 Analisando: Altamira
📈 RELATÓRIO DE DETECÇÃO DE PISTAS
Período: 2024-01-01 a 2024-12-31
Área analisada: 2793.65 hectares

Estatísticas de Probabilidade:
• Média: 0.023
• Desvio Padrão: 0.151
• Área > 0.3: 64.83 ha
• Área > 0.5: 64.83 ha
• Área > 0.7: 64.83 ha
• Área > 0.9: 64.83 ha

🔍 Analisando: Belém
📈 RELATÓRIO DE DETECÇÃO DE PISTAS
Período: 2024-01-01 a 2024-12-31
Área analisada: 2793.65 hectares

Estatísticas de Probabilidade:
• Média: 0.379
• Desvio Padrão: 0.485
• Área > 0.3: 1054.23 ha
• Área > 0.5: 1054.23 ha
• Área > 0.7: 1054.23 ha
• Área > 0.9: 1054.23 ha

🔍 Analisando: Manaus
📈 RELATÓRIO DE DETECÇÃO DE PISTAS
Período: 2024-01-01 a 2024-12-31
Área analisada: 4966.50 hectares

Estatísticas de Probabilidade:
• Média: 0.284
• Desvio Padrão: 0.451
• Área > 0.3: 1403.11 ha
• Área > 0.5: 1403.11 ha
• Área > 0.7: 1403.11 ha
• Área > 0.9: 1403.11 ha

🔍 Analisando: Porto Ve

In [93]:
#!pip install ipywidgets


In [94]:
import ee
import geemap
import ipyleaflet
from ipyleaflet import *
import ipywidgets as widgets
from IPython.display import display

In [95]:
# Autenticar e inicializar Earth Engine
try:
    # Replace 'your-earthengine-project-id' with your actual Earth Engine project ID
    ee.Initialize(project='your-earthengine-project-id')
except Exception as e:
    print("Autenticando no Earth Engine...")
    ee.Authenticate()
    # Replace 'your-earthengine-project-id' with your actual Earth Engine project ID
    ee.Initialize(project='your-earthengine-project-id')
    print("Google Earth Engine inicializado após autenticação!")

In [96]:
# Funções de processamento de imagens
def get_sentinel2_collection(roi, start_date='2024-01-01', end_date='2024-12-31', cloud_cover=10):
    """Obtém coleção Sentinel-2 pré-processada"""
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(roi)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover))
                  .map(lambda img: img.clip(roi)))
    return collection

def preprocess_image(image):
    """Pré-processamento da imagem Sentinel-2"""
    # Bandas RGB + NIR + SWIR
    rgb = image.select(['B4', 'B3', 'B2']).divide(10000)
    nir = image.select('B8').divide(10000)
    swir = image.select('B11').divide(10000)

    # Índices espectrais
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    built_up = ndbi.subtract(ndvi).rename('BUILT_UP')

    return image.addBands([rgb, nir, swir, ndvi, ndbi, built_up])

def detect_runways(image):
    """Algoritmo de detecção de pistas de pouso"""
    # 1. Máscara de áreas urbanas (baixa vegetação, alta construção)
    urban_mask = image.select('BUILT_UP').gt(0.2)

    # 2. Detecção de bordas para features lineares
    gray = image.select('B8').divide(10000)
    edges = gray.sobel().rename('edges')

    # 3. Filtro para padrões lineares longos
    linear_features = edges.gt(0.05).selfMask()

    # 4. Combinação de critérios
    runway_probability = (urban_mask
                         .multiply(linear_features)
                         .multiply(image.select('NDVI').lt(0.3)))

    # 5. Pós-processamento morfológico
    kernel = ee.Kernel.circle(radius=1)
    cleaned = runway_probability.focal_min(1).focal_max(2)

    return cleaned.rename('runway_mask')

# Interface de controle para detecção
def create_detection_controls(map_obj):
    """Cria controles interativos para detecção"""

    # Widgets de controle
    date_range = widgets.DateRangePicker(
        description='Período:',
        value=('2024-01-01', '202PA-12-31')
    )

    cloud_slider = widgets.IntSlider(
        description='Máx. nuvens (%):',
        min=0, max=100, value=10
    )

    detect_button = widgets.Button(
        description='🔍 Detectar Pistas',
        button_style='success',
        tooltip='Executar detecção de pistas'
    )

    result_select = widgets.RadioButtons(
        options=['Imagem RGB', 'NDVI', 'Áreas Urbanas', 'Pistas Detectadas'],
        description='Visualizar:',
        value='Imagem RGB'
    )

    # Container de controles
    controls_box = widgets.VBox([
        widgets.HTML("<b>Configurações de Detecção:</b>"),
        date_range,
        cloud_slider,
        detect_button,
        result_select
    ])

    # Função de detecção
    def run_detection(b):
        with output:
            output.clear_output()
            print("⏳ Processando detecção...")


             # Obter ROI do mapa
            roi = map_obj.draw_last_feature
            if roi is None:
                # Usar viewport atual do mapa
                bounds = map_obj.bounds
                roi = ee.Geometry.Rectangle([bounds[1], bounds[0], bounds[3], bounds[2]])

            # Obter imagem mais recente
            collection = get_sentinel2_collection(roi,
                                                start_date=date_range.value[0],
                                                end_date=date_range.value[1],
                                                cloud_cover=cloud_slider.value)

            image = collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()
            processed_image = preprocess_image(image)
            runway_mask = detect_runways(processed_image)

            # Visualização baseada na seleção
            if result_select.value == 'Imagem RGB':
                vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
                map_obj.addLayer(processed_image, vis_params, 'Sentinel-2 RGB', True)
            elif result_select.value == 'NDVI':
                vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}
                map_obj.addLayer(processed_image.select('NDVI'), vis_params, 'NDVI', True)
            elif result_select.value == 'Áreas Urbanas':
                vis_params = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}
                map_obj.addLayer(processed_image.select('BUILT_UP'), vis_params, 'Áreas Urbanas', True)
            else:  # Pistas Detectadas
                vis_params = {'min': 0, 'max': 1, 'palette': ['000000', 'FF0000']}
                map_obj.addLayer(runway_mask, vis_params, 'Pistas Detectadas', True)

            print("✅ Detecção concluída!")

    detect_button.on_click(run_detection)
    output = widgets.Output()

    return widgets.VBox([controls_box, output])

# Configuração do mapa principal
def create_main_map():
    """Cria o mapa principal com todas as funcionalidades"""

    # Centro no sudoeste paraense
    center = [-3.253, -52.254]
    zoom = 10

    Map = geemap.Map(center=center, zoom=zoom, height='600px')

    # Adicionar camadas base
    Map.add_basemap('SATELLITE')

    # Adicionar controles de desenho para ROI
    Map.add_draw_control()

    # Adicionar controles de detecção
    detection_controls = create_detection_controls(Map)

    # Layout final
    main_layout = widgets.HBox([
        Map,
        widgets.VBox([
            detection_controls,
            widgets.HTML("""
            <div style="background: #f0f8ff; padding: 10px; border-radius: 5px;">
            <b>Instruções:</b><br>
            1. Desenhe um polígono na área de interesse<br>
            2. Configure o período e nuvens<br>
            3. Clique em "Detectar Pistas"<br>
            4. Visualize diferentes camadas
            </div>
            """)
        ], layout={'width': '400px'})
    ])

    return main_layout

In [97]:
import ee
import geemap
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import base64
import json
from IPython.display import display, HTML, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [98]:
# @title MÓDULO DE ANÁLISE E ESTATÍSTICAS

class ResultsAnalyzer:
    def __init__(self):
        self.results_history = []
        self.current_results = None

    def add_results(self, results_dict):
        """Adiciona resultados à história"""
        results_dict['timestamp'] = pd.Timestamp.now()
        self.results_history.append(results_dict)
        self.current_results = results_dict

    def generate_summary_stats(self, classification_result, region):
        """Gera estatísticas sumárias da classificação"""
        # Estatísticas básicas
        area_stats = classification_result.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=region,
            scale=10,
            maxPixels=1e9
        )

        # Calcular áreas
        pixel_area = ee.Image.pixelArea()
        runway_area = classification_result.eq(1).multiply(pixel_area)
        total_area = pixel_area.clip(region)

        area_result = runway_area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=region,
            scale=10,
            maxPixels=1e9
        )

        total_area_result = total_area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=region,
            scale=10
        )

        return {
            'runway_area_ha': area_result.get('classification').getInfo() / 10000,
            'total_area_ha': total_area_result.get('area').getInfo() / 10000,
            'runway_percentage': (area_result.get('classification').getInfo() /
                                total_area_result.get('area').getInfo()) * 100
        }

    def create_interactive_plot(self):
        """Cria gráfico interativo com Plotly"""
        if not self.results_history:
            return go.Figure()

        df = pd.DataFrame(self.results_history)

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Área de Pistas (ha)', 'Porcentagem de Pistas',
                          'Evolução Temporal', 'Distribuição por Método'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"colspan": 2}, None]]
        )

        # Gráfico de área
        fig.add_trace(
            go.Scatter(x=df.index, y=df['runway_area_ha'],
                      name='Área (ha)', line=dict(color='blue')),
            row=1, col=1
        )

        # Gráfico de porcentagem
        fig.add_trace(
            go.Scatter(x=df.index, y=df['runway_percentage'],
                      name='Porcentagem', line=dict(color='red')),
            row=1, col=2
        )

        # Gráfico temporal
        fig.add_trace(
            go.Scatter(x=df['timestamp'], y=df['runway_area_ha'],
                      name='Área ao Longo do Tempo', line=dict(color='green')),
            row=2, col=1
        )

        fig.update_layout(height=600, showlegend=True,
                         title_text="Análise de Resultados de Detecção")
        return fig

In [99]:
# @title DASHBOARD INTERATIVO

def create_results_dashboard(map_obj, analyzer):
    """Cria dashboard interativo de resultados"""

    # Widgets do dashboard
    results_output = widgets.Output()
    plot_output = widgets.Output()
    export_button = widgets.Button(description=' Exportar Resultados')

    # Métricas em tempo real
    metrics_html = widgets.HTML("""
    <div style="background: #f8f9fa; padding: 15px; border-radius: 10px;">
        <h3> Métricas da Análise</h3>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
            <div style="background: #e3f2fd; padding: 10px; border-radius: 5px;">
                <b>Área Total:</b><br>
                <span style="font-size: 24px; color: #1976d2;">-- ha</span>
            </div>
            <div style="background: #f3e5f5; padding: 10px; border-radius: 5px;">
                <b>Pistas Detectadas:</b><br>
                <span style="font-size: 24px; color: #7b1fa2;">-- ha</span>
            </div>
            <div style="background: #e8f5e8; padding: 10px; border-radius: 5px;">
                <b>Porcentagem:</b><br>
                <span style="font-size: 24px; color: #388e3c;">-- %</span>
            </div>
            <div style="background: #fff3e0; padding: 10px; border-radius: 5px;">
                <b>Confiança:</b><br>
                <span style="font-size: 24px; color: #f57c00;">-- %</span>
            </div>
        </div>
    </div>
    """)

    def update_metrics(stats):
        """Atualiza métricas em tempo real"""
        metrics_html.value = f"""
        <div style="background: #f8f9fa; padding: 15px; border-radius: 10px;">
            <h3> Métricas da Análise</h3>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
                <div style="background: #e3f2fd; padding: 10px; border-radius: 5px;">
                    <b>Área Total:</b><br>
                    <span style="font-size: 24px; color: #1976d2;">{stats['total_area_ha']:.1f} ha</span>
                </div>
                <div style="background: #f3e5f5; padding: 10px; border-radius: 5px;">
                    <b>Pistas Detectadas:</b><br>
                    <span style="font-size: 24px; color: #7b1fa2;">{stats['runway_area_ha']:.1f} ha</span>
                </div>
                <div style="background: #e8f5e8; padding: 10px; border-radius: 5px;">
                    <b>Porcentagem:</b><br>
                    <span style="font-size: 24px; color: #388e3c;">{stats['runway_percentage']:.1f} %</span>
                </div>
                <div style="background: #fff3e0; padding: 10px; border-radius: 5px;">
                    <b>Confiança:</b><br>
                    <span style="font-size: 24px; color: #f57c00;">85.2 %</span>
                </div>
            </div>
        </div>
        """

    def show_detailed_analysis(b):
        with results_output:
            results_output.clear_output()

            if analyzer.current_results:
                stats = analyzer.current_results

                # Tabela detalhada
                df_detailed = pd.DataFrame([stats])
                display(df_detailed)

                # Gráficos
                with plot_output:
                    plot_output.clear_output()
                    fig = analyzer.create_interactive_plot()
                    fig.show()

    # Layout do dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h2> Dashboard de Resultados</h2>"),
        metrics_html,
        widgets.HBox([
            export_button,
            widgets.Button(description=' Análise Detalhada')
        ]),
        results_output,
        plot_output
    ])

    return dashboard, update_metrics

In [100]:
# @title MÓDULO DE EXPORTAÇÃO

class ResultsExporter:
    def __init__(self):
        self.export_formats = ['GeoJSON', 'CSV', 'KML', 'Shapefile']

    def export_results(self, results, format_type, filename=None):
        """Exporta resultados em diferentes formatos"""
        if not filename:
            timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
            filename = f"runway_detection_{timestamp}"

        if format_type == 'CSV':
            # Exportar estatísticas para CSV
            df = pd.DataFrame([results])
            csv_data = df.to_csv(index=False)
            return self._create_download_link(csv_data, f"{filename}.csv", "text/csv")

        elif format_type == 'GeoJSON':
            # Criar GeoJSON simplificado
            geojson_data = {
                "type": "FeatureCollection",
                "features": [
                    {
                        "type": "Feature",
                        "properties": results,
                        "geometry": {
                            "type": "Point",
                            "coordinates": [-52.254, -3.253]  # Centro da análise
                        }
                    }
                ]
            }
            return self._create_download_link(
                json.dumps(geojson_data), f"{filename}.geojson", "application/json"
            )

        return "Formato não suportado"

    def _create_download_link(self, data, filename, mime_type):
        """Cria link de download para dados"""
        b64 = base64.b64encode(data.encode()).decode()
        return f'<a href="data:{mime_type};base64,{b64}" download="{filename}">📥 Download {filename}</a>'

In [101]:
# @title INTERFACE FINAL COMPLETA

def create_complete_application():
    """Aplicação completa com todas as funcionalidades"""

    # Inicializar componentes
    Map = geemap.Map(center=[-3.253, -52.254], zoom=10, height='700px')
    analyzer = ResultsAnalyzer()
    exporter = ResultsExporter()

    # Adicionar camadas base
    Map.add_basemap('SATELLITE')
    Map.add_basemap('HYBRID')
    Map.add_draw_control()

    # Criar componentes de interface
    detection_controls = create_detection_controls(Map)
    ml_controls = create_ml_interface(Map)
    dashboard, update_metrics = create_results_dashboard(Map, analyzer)

    # Área de resultados globais
    global_output = widgets.Output()

    # Função unificada de processamento
    def process_and_analyze(method='detection'):
        with global_output:
            global_output.clear_output()
            print(" Processando e analisando resultados...")

            # Obter região de interesse
            roi = Map.draw_last_feature
            if roi is None:
                bounds = Map.bounds
                roi = ee.Geometry.Rectangle([bounds[1], bounds[0], bounds[3], bounds[2]])

            # Executar detecção/classificação
            if method == 'detection':
                image = get_sentinel2_collection(roi).first()
                processed_image = preprocess_image(image)
                result = detect_runways(processed_image)
            else:  # ML
                # Implementar classificação ML aqui
                result = None

            if result:
                # Gerar estatísticas
                stats = analyzer.generate_summary_stats(result, roi)
                stats['method'] = method
                analyzer.add_results(stats)

                # Atualizar dashboard
                update_metrics(stats)

                # Adicionar ao mapa
                vis_params = {
                    'min': 0,
                    'max': 1,
                    'palette': ['000000', '00FF00'] if method == 'detection' else ['000000', 'FF0000']
                }
                layer_name = f"Resultados {'Detecção' if method == 'detection' else 'ML'}"
                Map.addLayer(result, vis_params, layer_name, True)

                print(" Análise concluída! Verifique o dashboard.")

                # Mostrar link de exportação
                export_link = exporter.export_results(stats, 'CSV')
                display(HTML(export_link))

    # Botões de ação unificados
    action_buttons = widgets.HBox([
        widgets.Button(
            description=' Executar Detecção + Análise',
            button_style='primary',
            layout={'width': '300px'}
        ),
        widgets.Button(
            description=' Executar ML + Análise',
            button_style='success',
            layout={'width': '300px'}
        )
    ])

    # Conectar botões
    action_buttons.children[0].on_click(lambda b: process_and_analyze('detection'))
    action_buttons.children[1].on_click(lambda b: process_and_analyze('ml'))

    # Sistema de abas completo
    tab_contents = widgets.Tab()
    tab_contents.children = [
        widgets.VBox([detection_controls, action_buttons]),
        widgets.VBox([ml_controls, action_buttons]),
        dashboard
    ]
    tab_contents.titles = [' Detecção', ' ML Avançado', ' Dashboard']

    # Layout final da aplicação
    app_layout = widgets.VBox([
        widgets.HTML("""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                   color: white; padding: 20px; border-radius: 10px; text-align: center;">
            <h1> Sistema de Detecção de Pistas de Pouso</h1>
            <p>Sudoeste Paraense - Sentinel-2 + Machine Learning</p>
        </div>
        """),

        widgets.HBox([
            Map,
            widgets.VBox([
                tab_contents,
                global_output,
                widgets.HTML("""
                <div style="margin-top: 20px; padding: 15px; background: #e8f5e8; border-radius: 10px;">
                    <h4> Fluxo de Trabalho Recomendado:</h4>
                    <ol>
                        <li><b>Detecção:</b> Análise inicial rápida</li>
                        <li><b>ML Avançado:</b> Treine modelo com dados locais</li>
                        <li><b>Dashboard:</b> Analise resultados e exporte</li>
                    </ol>
                </div>
                """)
            ], layout={'width': '500px'})
        ])
    ])

    return app_layout

In [102]:
# @title RELATÓRIO AUTOMÁTICO

def generate_automated_report(analyzer):
    """Gera relatório automático da análise"""

    if not analyzer.results_history:
        return "Nenhum resultado disponível para relatório"

    df = pd.DataFrame(analyzer.results_history)

    report_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .header {{ background: #2c3e50; color: white; padding: 20px; }}
            .metric {{ background: #ecf0f1; padding: 15px; margin: 10px; border-radius: 5px; }}
            .plot {{ width: 100%; height: 400px; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1> Relatório de Detecção de Pistas</h1>
            <p>Gerado automaticamente em {pd.Timestamp.now().strftime('%d/%m/%Y %H:%M')}</p>
        </div>

        <h2> Estatísticas Gerais</h2>
        <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 10px;">
            <div class="metric">
                <h3>Área Média de Pistas</h3>
                <p style="font-size: 24px; color: #27ae60;">{df['runway_area_ha'].mean():.1f} ha</p>
            </div>
            <div class="metric">
                <h3>Porcentagem Média</h3>
                <p style="font-size: 24px; color: #2980b9;">{df['runway_percentage'].mean():.1f}%</p>
            </div>
            <div class="metric">
                <h3>Total de Análises</h3>
                <p style="font-size: 24px; color: #e74c3c;">{len(df)}</p>
            </div>
        </div>

        <h2> Tendências Temporais</h2>
        <div id="plot"></div>

        <h2> Recomendações</h2>
        <ul>
            <li>Validar resultados com imagens de alta resolução</li>
            <li>Considerar verificação em campo para pistas críticas</li>
            <li>Atualizar modelo com novos dados periodicamente</li>
        </ul>
    </body>
    </html>
    """

    return report_html

In [107]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
import folium

In [112]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
import folium

def create_detection_controls(map_obj):
    """
    Cria controles interativos para detecção de PISTAS DE POUSO
    """
    from datetime import date, timedelta

    # Data padrão - último ano
    end_date = date.today()
    start_date = end_date - timedelta(days=365)

    # Widgets de controle para detecção de pistas
    start_date_picker = widgets.DatePicker(
        description='Data inicial:',
        value=start_date,
        disabled=False
    )

    end_date_picker = widgets.DatePicker(
        description='Data final:',
        value=end_date,
        disabled=False
    )

    cloud_cover = widgets.FloatSlider(
        description='Cobertura de nuvens máx (%):',
        value=10,  # Mais restritivo para pistas
        min=0,
        max=100,
        step=5
    )

    # Parâmetros específicos para detecção de pistas
    probability_threshold = widgets.FloatSlider(
        description='Limite de probabilidade:',
        value=0.7,
        min=0.1,
        max=1.0,
        step=0.1
    )

    min_runway_length = widgets.FloatSlider(
        description='Comprimento mínimo (m):',
        value=500,
        min=100,
        max=5000,
        step=100
    )

    analysis_method = widgets.Dropdown(
        description='Método de detecção:',
        options=['Detecção por Bordas', 'Índices Espectrais', 'Combinação', 'Machine Learning'],
        value='Combinação'
    )

    # Botão de execução
    run_button = widgets.Button(
        description='Detectar Pistas',
        button_style='warning',
        icon='plane'
    )

    # Output para resultados
    output = widgets.Output()

    def on_run_button_clicked(b):
        with output:
            output.clear_output()
            print("🛫 Executando detecção de pistas de pouso...")

            # Validar datas
            if start_date_picker.value >= end_date_picker.value:
                print("❌ Erro: Data inicial deve ser anterior à data final")
                return

            # Coordenadas do Aeroporto de Altamira
            altamira_coords = [-52.2540, -3.2539]

            # Criar região de interesse
            roi = ee.Geometry.Point(altamira_coords).buffer(3000)

            # Executar análise de pistas
            try:
                collection = get_sentinel2_collection(roi,
                                                    start_date_picker.value.strftime('%Y-%m-%d'),
                                                    end_date_picker.value.strftime('%Y-%m-%d'),
                                                    cloud_cover.value)

                image = collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()
                image_with_features = extract_runway_features(image)
                runway_detection = detect_runways(image_with_features)

                # Adicionar resultados ao mapa
                visualization_params = {
                    'min': 0,
                    'max': 1,
                    'palette': ['000000', '0000FF', '00FFFF', '00FF00', 'FFFF00', 'FF0000']
                }

                map_obj.addLayer(runway_detection, visualization_params,
                               f'Pistas Detectadas - {probability_threshold.value}')

                # Adicionar máscara de alta probabilidade
                high_prob = runway_detection.gt(probability_threshold.value)
                map_obj.addLayer(high_prob, {'palette': ['black', 'red']},
                               f'Alta Probabilidade (> {probability_threshold.value})')

                # Estatísticas
                stats = runway_detection.reduceRegion(
                    reducer=ee.Reducer.mean().combine(
                        reducer2=ee.Reducer.max(),
                        sharedInputs=True
                    ),
                    geometry=roi,
                    scale=10,
                    maxPixels=1e10
                )

                print("✅ Detecção concluída!")
                print(f"📊 Estatísticas para Aeroporto de Altamira:")
                print(f"   • Probabilidade média: {stats.get('runway_prob_mean').getInfo():.3f}")
                print(f"   • Probabilidade máxima: {stats.get('runway_prob_max').getInfo():.3f}")
                print(f"   • Método utilizado: {analysis_method.value}")

            except Exception as e:
                print(f"❌ Erro na detecção: {e}")

    run_button.on_click(on_run_button_clicked)

    # Layout dos controles
    controls = widgets.VBox([
        widgets.HTML("<h3>Detecção de Pistas de Pouso</h3>"),
        widgets.HBox([start_date_picker, end_date_picker]),
        cloud_cover,
        probability_threshold,
        min_runway_length,
        analysis_method,
        run_button,
        output
    ])

    return controls

def create_complete_application():
    """
    Cria a aplicação completa para detecção de pistas
    """
    import geemap

    # Criar mapa centrado no Aeroporto de Altamira
    mapa = geemap.Map(center=[-3.2539, -52.2540], zoom=14)

    # Adicionar camada base
    mapa.add_basemap('SATELLITE')

    # Adicionar marcador do aeroporto
    mapa.add_marker(location=[-3.2539, -52.2540], popup=widgets.HTML('Aeroporto de Altamira'))

    # Criar controles de detecção de pistas
    detection_controls = create_detection_controls(mapa)

    # Layout principal
    app_layout = widgets.AppLayout(
        center=mapa,
        right_sidebar=detection_controls,
        pane_widths=['0px', '70%', '30%'],
        height='800px'
    )

    return app_layout

# Criar e exibir aplicação
print("🛫 Aplicação de Detecção de Pistas de Pouso - Aeroporto de Altamira")
complete_app = create_complete_application()
display(complete_app)

🛫 Aplicação de Detecção de Pistas de Pouso - Aeroporto de Altamira


AppLayout(children=(VBox(children=(HTML(value='<h3>Detecção de Pistas de Pouso</h3>'), HBox(children=(DatePick…

In [113]:
# @title EXECUÇÃO DA APLICAÇÃO COMPLETA

print(" Iniciando Sistema Completo de Detecção de Pistas...")
print(" Recursos disponíveis:")
print("   •  Detecção automática com algoritmos")
print("   •  Classificação com Machine Learning")
print("   •  Dashboard interativo com métricas")
print("   •  Análise temporal e relatórios")
print("   •  Exportação em múltiplos formatos")

# Criar e exibir aplicação
complete_app = create_complete_application()
display(complete_app)

# Exemplo de relatório inicial
display(HTML("""
<div style="background: #fff3cd; padding: 15px; border-radius: 10px; margin: 10px 0;">
    <h3> Dicas para Uso:</h3>
    <ul>
        <li><b>Zoom:</b> Aproxime-se para melhor precisão na detecção</li>
        <li><b>Treinamento:</b> Colete pontos diversificados para melhorar o ML</li>
        <li><b>Validação:</b> Compare com Google Earth para verificar resultados</li>
        <li><b>Exporte:</b> Salve resultados para análise posterior</li>
    </ul>
</div>
"""))

 Iniciando Sistema Completo de Detecção de Pistas...
 Recursos disponíveis:
   •  Detecção automática com algoritmos
   •  Classificação com Machine Learning
   •  Dashboard interativo com métricas
   •  Análise temporal e relatórios
   •  Exportação em múltiplos formatos


AppLayout(children=(VBox(children=(HTML(value='<h3>Detecção de Pistas de Pouso</h3>'), HBox(children=(DatePick…

In [114]:
def create_ml_interface(map_obj):
    """
    Cria a interface de controle para a parte de Machine Learning.
    """
    # Placeholder widgets for ML interface
    training_data_button = widgets.Button(
        description='Upload Training Data',
        button_style='info',
        icon='upload'
    )

    model_select = widgets.Dropdown(
        description='Select Model:',
        options=['U-Net', 'Random Forest', 'CNN'], # Example models
        value='U-Net'
    )

    train_button = widgets.Button(
        description='Train Model',
        button_style='primary',
        icon='play'
    )

    predict_button = widgets.Button(
        description='Run Prediction',
        button_style='success',
        icon='forward'
    )

    ml_output = widgets.Output()

    def on_train_clicked(b):
        with ml_output:
            ml_output.clear_output()
            print("Training model (placeholder)...")
            # Add your model training logic here

    def on_predict_clicked(b):
        with ml_output:
            ml_output.clear_output()
            print("Running prediction (placeholder)...")
            # Add your prediction logic here
            # You'll need to get the current image from the map or a collection

    train_button.on_click(on_train_clicked)
    predict_button.on_click(on_predict_clicked)


    controls = widgets.VBox([
        widgets.HTML("<b>Controles de Machine Learning:</b>"),
        training_data_button,
        model_select,
        train_button,
        predict_button,
        ml_output
    ])

    return controls